In [ ]:
import numpy as np
import random
from bs4 import BeautifulSoup
import pandas as pd
from pm4py.objects.log.util import dataframe_utils
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.algo.discovery.inductive import algorithm as inductive_miner
from pm4py.visualization.process_tree import visualizer as pt_visualizer
from pm4py.visualization.dfg import visualizer as dfg_visualization
from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
import matplotlib.pyplot as plt
import networkx as nx
from colour import Color
import copy
from collections import Counter

from networkx.drawing.nx_agraph import graphviz_layout, to_agraph
import pygraphviz as pgv

import json

import seaborn as sns

np.random.seed(42)
random.seed(42)
import pm4py

from collections import Counter
import subprocess

In [ ]:
from pm4py.objects.log.importer.xes import importer as xes_importer
log_application = xes_importer.apply('BPI Challenge 2017.xes')

In [ ]:
# A_complete is always preceeded by "W_call after offers"
for trace in log_application:
    for pos_index in range(len(trace)):
        pos = trace[pos_index]
        if "A_Complete" in pos["concept:name"]:
            if "W_Call after offers" not in trace[pos_index-1]["concept:name"]:
                for e in trace[pos_index-5:pos_index+5]:
                    print(e["concept:name"])
                print("not followed")
                assert(False)

In [ ]:
# "W_call after offers" has always "A_complete"
for trace in log_application:
    for pos_index in range(len(trace)):
        pos = trace[pos_index]
        if "W_call after offers" in pos["concept:name"]:
            index_inner = pos_index+1
            while trace[index_inner]["concept:name"] == "W_call after offers":
                index_inner += 1
            if "A_complete" not in trace[index_inner]["concept:name"]:
                for e in trace[index_inner-5:index_inner+5]:
                    print(e["concept:name"])
                print("not followed")
                assert(False)

In [ ]:
# A_Incomplete is always surrounded by "W_Call incomplete files"
for trace in log_application:
    for pos_index in range(len(trace)):
        pos = trace[pos_index]
        if "A_Incomplete" in pos["concept:name"]:
            if "W_Call incomplete files" not in trace[pos_index-1]["concept:name"] and "W_Call incomplete files" not in trace[pos_index+1]["concept:name"]:
                for e in trace[pos_index-5:pos_index+5]:
                    print(e["concept:name"])
                print("not followed")
                assert(False)

In [ ]:
#Create offer is always followed by "Created"
for trace in log_application:
    for pos_index in range(len(trace)):
        pos = trace[pos_index]
        if "Create Offer" in pos["concept:name"]:
            if "Created" not in trace[pos_index+1]["concept:name"]:
                print("not followed")

In [ ]:
# number of calls
for trace in log_application:
    for pos_index in range(len(trace)):
        pos = trace[pos_index]
        if "Create Offer" in pos["concept:name"]:
            if "Created" not in trace[pos_index+1]["concept:name"]:
                print("not followed")

In [ ]:
trace = log_application[76]
for i in range(len(trace)):
    if "W_" not in trace[i]['concept:name'] or True:
        print(trace[i]['concept:name'], trace[i]['lifecycle:transition'], trace[i]['time:timestamp'])
print("###########")
for i in range(len(trace)):
    if "W_" in trace[i]['concept:name']:
        print(trace[i]['concept:name'], trace[i]['lifecycle:transition'], trace[i]['time:timestamp'])

In [ ]:
def contains(trace, element):
    for event in trace:
        if event['concept:name']==element:
            return True
        
count = 0
positive = 0
negative = 0
found_indices = []
for i in range(len(log_application)):
    trace = log_application[i]
    found = False
    for pos in trace:
        if "Call" in pos['concept:name'] and pos['lifecycle:transition'] == "complete":
            found = True
    if found:
        print(i)
        found_indices.append(i)
        count += 1
        if contains(trace, 'A_Pending'):
            positive += 1
        else:
            negative += 1
        continue
print("found completed call events",count)
print("pos", positive)
print("neg", negative)

In [ ]:
times = {}
for i in range(len(log_application)):
    trace = log_application[i]
    for index in range(len(trace)):
        pos = trace[index]
        if "Call" in pos['concept:name'] and pos['lifecycle:transition'] in ["start", "resume"]:
            for index_inner in range(index+1, len(trace)):
                pos_inner = trace[index_inner]
                if "Call" in pos_inner['concept:name'] and pos_inner['lifecycle:transition'] in ["complete", "suspend"]:
                    if i not in times:
                        times[i] = []
                    times[i].append((pos_inner["time:timestamp"] - pos["time:timestamp"]).seconds)
                    # computes from start/resume until next end position
                    break

In [ ]:
completed = []
uncompleted = []
for i in found_indices:
    for e in times[i]:
        completed.append(e)
print("not in")
for i in times:
    if i not in found_indices:
        for e in times[i]:
            uncompleted.append(e)
plt.hist(completed, 1000)
plt.show()
plt.hist(uncompleted, 1000)
plt.show()
print(max(uncompleted))

In [ ]:
started = []
started_time = {}
closed_trace = []
for index in range(len(trace)):
    pos = trace[index]
    event = pos["concept:name"]
    lifecycle = pos['lifecycle:transition']
    if "W_" not in event:
        closed_trace.append(pos)
    elif "Call" in event:
        if lifecycle == ["start"]:
            closed = False
            for index_inner in range(index+1, len(trace)):
                pos_inner = trace[index_inner]
                if pos_inner["concept:name"] == event:
                    if pos_inner["lifecycle:transition"] == "closed":
                        
                elif:
                    break
        if event not in started:
            started.append(pos["concept:name"])
            started_time[pos["concept:name"]] = pos["time:timestamp"]
    elif pos['lifecycle:transition'] in ["resume", "start"] and "Call" in pos['concept:name']:
        if pos["concept:name"] in started:
    else:
        closed_trace.append(pos)
print(closed_trace)

In [ ]:
log = copy.deepcopy(log_application)
log_before = pm4py.filter_time_range(log, "2011-03-09 00:00:00", "2016-06-30 23:59:59", mode='traces_contained')
log_after = pm4py.filter_time_range(log, "2016-08-01 00:00:00", "2018-03-09 00:00:00", mode='traces_contained')
print(np.shape(log))
print(np.shape(log_before))
print(np.shape(log_after))

In [ ]:
perc = 2/len(log_before)
filtered_log = pm4py.filter_variants_by_coverage_percentage(log_before, perc)
print(len(filtered_log))

In [ ]:
TIME_DIFF = 0
MIN_SPEAKING_TIME = 60
SPEAK_THRESHOLD = 600
DAY_DIFFERENCE = 20
durations = {}
def construct_log(log):
    terminal_states = ['A_Cancelled COMPANY', 'A_Cancelled CUSTOMER', 'A_Pending']#, 'A_Denied']
    to_merge = ['W_Call incomplete files', 'W_Call after offers', 'W_Complete application', 'W_Validate application']
    log_activities = []
    for trace in log:
        current_trace = [trace[0]]
        for i in range(1,len(trace)):
            pos = trace[i]
            if "W_Call" in trace[i]['concept:name']:
                # search for closing event
                if pos['lifecycle:transition'] in ["start", "resume"]:
                    for inner_index in range(i+1, len(trace)):
                        inner_pos = trace[inner_index]
                        if pos['concept:name'] == inner_pos['concept:name']:
                            if inner_pos['lifecycle:transition'] in ["suspend", "complete"]:                 
                                duration = (inner_pos['time:timestamp']-pos['time:timestamp']).total_seconds()
                                if duration > MIN_SPEAKING_TIME:
                                    if duration > 10000:
                                        print(i, inner_index)
                                        print(pos)
                                        print(inner_pos)
                                        print(trace[i: inner_index+1])
                                        print("#########")
                                    if pos['concept:name'] in current_trace[-1]["concept:name"]:
                                        current_trace[-1]["duration"] += duration
                                    else:
                                        current_trace.append(pos)
                                        current_trace[-1]['duration'] = duration
                                    if current_trace[-1]["duration"] < SPEAK_THRESHOLD:
                                        current_trace[-1]['concept:name'] = pos['concept:name']+" SHORT"
                                    elif current_trace[-1]["duration"] < 14400:
                                        current_trace[-1]['concept:name'] = pos['concept:name']+" LONG"
                                    else:
                                        current_trace[-1]['concept:name'] = pos['concept:name']+" SUPER LONG"
                                    if pos['concept:name'] not in durations:
                                        durations[trace[i]['concept:name']] = []
                                    durations[trace[i]['concept:name']].append(duration)
                            break
            if "W_" in trace[i]['concept:name']:
                continue
            if trace[i]['concept:name'] in ["A_Created", "A_Complete", "A_Incomplete"]:
                continue
            if trace[i]['concept:name'] == "A_Cancelled":
                current_trace.append(pos)
                if (trace[i]['time:timestamp']-trace[i-1]['time:timestamp']).days > DAY_DIFFERENCE:
                    current_trace[-1]['concept:name'] += " COMPANY"
                else:
                    current_trace[-1]['concept:name'] += " CUSTOMER"
                continue
            if "O_Created" == trace[i]['concept:name']:
                continue # merge create and created
            """
            if "A_Cancelled" == trace[i]['concept:name']:
                current_trace.append(pos)
                if (trace[i]['time:timestamp']-trace[i-1]['time:timestamp']).days >28:
                    current_trace[-1]['concept:name'] = current_trace[-1]['concept:name']+" AUTO"
                else:
                    current_trace[-1]['concept:name'] = current_trace[-1]['concept:name']+" USER"
            """
            if trace[i]['concept:name'] in terminal_states:
                current_trace.append(pos)
            else:
                if trace[i]['concept:name'] in to_merge and trace[i]['concept:name'] == trace[i-1]['concept:name']:
                    continue
                elif (trace[i]['time:timestamp']-trace[i-1]['time:timestamp']).total_seconds()>TIME_DIFF:
                    current_trace.append(pos)
        if "A_Pending" in [pos['concept:name'] for pos in current_trace]:
            if "O_Cancelled" in [pos['concept:name'] for pos in current_trace]:
                for pos1 in current_trace:
                    if 'O_Cancelled' in pos1['concept:name']:
                        current_trace.remove(pos1)
        intersection = [i for i in trace if i['concept:name'] in terminal_states]
        for state in terminal_states:
            indices = [i for i, x in enumerate(current_trace) if x['concept:name'] == state]
            if indices:
                current_trace = current_trace[:indices[0]+1]
        if intersection:
            log_activities.append(current_trace)
    return log_activities
print(len(filtered_log))
filtered_log = construct_log(filtered_log)
print(len(filtered_log))

In [ ]:
duration_incomplete_list = []
duration_offer_list = []
for trace in filtered_log:
    duration_incomplete = 0
    duration_offer = 0
    for pos in trace:
        if "W_Call incomplete files" in pos["concept:name"]:
            duration_incomplete += pos["duration"]
        if "W_Call after offers" in pos["concept:name"]:
            duration_offer += pos["duration"]
    duration_incomplete_list.append(duration_incomplete)
    duration_offer_list.append(duration_offer)
plt.hist(duration_incomplete_list,100)
plt.show()
plt.hist(duration_offer_list,100)
plt.show()

In [ ]:
for trace in filtered_log:
    for pos in trace:
        if "duration" in pos.keys():
            if pos["duration"]>20000:
                print(pos["duration"])
                for pos in trace:
                    print(pos['concept:name'], pos['time:timestamp'])
                assert(False)

In [ ]:
print(len(durations))
after_offers = durations["W_Call after offers SHORT"]+durations["W_Call after offers LONG"]
incomplete_files = durations["W_Call incomplete files SHORT"]+durations["W_Call incomplete files LONG"]
print(max(after_offers))
print(max(incomplete_files))
print("W_Call after offers")
plt.hist(after_offers, 100)
plt.show()
print("W_Call incomplete files")
plt.hist(incomplete_files, 100)
plt.show()

In [ ]:
print(len(durations))
print("W_Call after offers short")
after_short = [x for x in after_offers if x < 1000]
incomplete_short = [x for x in incomplete_files if x < 1000]

plt.hist(after_short, 100)
plt.show()
print("W_Call incomplete files short")
plt.hist(incomplete_short, 100)
plt.show()

In [ ]:
events = set([x['concept:name'] for trace in log_application for x in trace])
print(events)
print(len(events))
print([e for e in events if "W_" in e])

In [ ]:
print(set([x['concept:name'] for trace in filtered_log for x in trace]))
print(len(set([x['concept:name'] for trace in filtered_log for x in trace])))

In [ ]:
def contains(trace, element):
    for event in trace:
        if event['concept:name']==element:
            return True
        
# process log to iterate created offers
MAX_INDEX = 100
def unroll_log(log):
    
    for trace in log:
        #trace.insert(0,{'concept:name': "start"})
        isPositive = False
        if contains(trace, 'A_Pending'):
            isPositive = True
        trace.insert(0,{'concept:name': "start"})
        if isPositive:
            trace.append({'concept:name': "positive"})
        else:
            trace.append({'concept:name': "negative"})
    
    to_extend = ["O_Create Offer", "O_Created", "W_Call after offers", "W_Call incomplete files", "W_Validate application", "O_Sent (mail and online)", "O_Sent (online only)"]
    to_extend = ["O_Create Offer"]
    for name in to_extend:
        element = 0
        max_trace = 0
        max_index = 0
        for trace in log:
            indices = [i for i, x in enumerate(trace) if x['concept:name'] == name]
            for i in indices:
                count_indices = [j for j in indices if j < i]
                index = MAX_INDEX if len(count_indices) > MAX_INDEX else len(count_indices)
                trace[i]['concept:name'] += " "+str(index)
                if len(count_indices)>max_index:
                    max_index = len(count_indices)
                    element = trace[i]['concept:name']
                    max_trace = trace
        print(element, max_trace)
    return log

unroll_log(filtered_log)

In [ ]:
def plot_lengths(log):
    lenghts = [len(x) for x in log]
    plt.hist(lenghts, len(set(lenghts)))
    plt.show()
plot_lengths(filtered_log)

In [ ]:
def plot_negative_positive(log):
    outcome = ["positive" if "positive" in [i['concept:name'] for i in trace] else "negative" for trace in log]
    print(Counter(outcome))
plot_negative_positive(filtered_log)

In [ ]:
REMOVE_THRESHOLD = 0

def ms(trace):
    multiset = {}
    for pos in trace:
        if pos['concept:name'] not in multiset:
            multiset[pos['concept:name']] = 1
        else:
            multiset[pos['concept:name']] += 1
    return json.dumps(multiset, sort_keys=True).encode()

def hist(trace): 
    hist = str(trace[0]['concept:name'])
    for pos in trace[1:]:
        hist += " - " + str(pos['concept:name'])
    return hist

def transition_system(log, threshold = 0, history = 1, actors = {}):
    edges = []
    edge_counter = {}
    controll = {}
    action = {}
    edge_mapping = {}
    for trace_index in range(len(log)):
        trace = log[trace_index]
        s = "start"
        assert(trace[0]['concept:name']=="start")
        for pos_index in range(1,len(trace)):
            pos = trace[pos_index]
            activity = pos['concept:name']
            controllable = True
            for key in actors:
                if key in activity:
                    controllable = actors[key]=='company'
            #t = ms(trace[max(0,pos_index-history+1):pos_index+1])
            t = hist(trace[max(0,pos_index-history+1):pos_index+1])
            e = (s,t)
            controll[e] = controllable
            action[e] = activity
            if e not in edges:
                edges.append(e)
                edge_counter[e] = 1
                edge_mapping[e] = [trace_index]
            else:
                edge_counter[e] = edge_counter[e]+1
                edge_mapping[e].append(trace_index)
            s = t
    g = nx.DiGraph()
    for e in edges:
        g.add_edge(e[0], e[1])
    to_remove = []
    trace_index_to_remove = []
    for e in g.edges:
        if e[0] == e[1]:
            to_remove.append(e)
        if edge_counter[e] <= threshold:
            for trace_index in edge_mapping[e]:
                trace_index_to_remove.append(trace_index)
        g[e[0]][e[1]]['controllable'] = controll[e]
        g[e[0]][e[1]]['action'] = action[e]

    for e in to_remove:
        if e in g.edges():
            g.remove_edge(e[0],e[1])
    print("numer of removed index", len(trace_index_to_remove))
    
    log_reduced = copy.deepcopy(log)
    for index in trace_index_to_remove:
        log_reduced.pop(index)
    
    if threshold>0:
        return transition_system(log_reduced, history = history, actors = actors)
    
    return g, edge_mapping

with open('activities.xml') as f:
    data = f.read()
actors = json.loads(data)

system, edge_mapping = transition_system(filtered_log, REMOVE_THRESHOLD, history = 3, actors = actors)
print(len(system.nodes))
print(len(system.edges))

In [ ]:
# compute weights
def isInTrace(s,t, trace):
    for i in range(len(trace)-1):
        if trace[i]['concept:name'] == s and trace[i+1]['concept:name'] == t:
            return True
    return False

def weight(trace):
    return 1 if any("positive" in pos['concept:name'] for pos in trace) else -1

def entropy(p1, p2):
    if p1 == 0 or p2 == 0:
        return 0
    return - p1*np.log2(p1) - p2* np.log2(p2)

def distribution(s,t,log, edge_mapping):
    distr = {1.0: 0 , -1.0 : 0}
    assert((s,t) in edge_mapping)
    for trace_index in edge_mapping[(s,t)]:
        w = weight(log[trace_index])
        distr[w] += 1 #
    return distr[1], distr[-1]

def compute_edge_cost(g, traces, edge_mapping):
    #weights = [1 if "finished" in i else -1 for i in traces]
    edge_cost = {}
    counter = 1
    for s in g.nodes:
        counter +=1
        for t in g[s]:
            #if s == "start" or s == "fin" or t == "start" or t == "fin":
                #continue
            #print("s:", s, "t:",t)
            
            p1, p2 = distribution(s,t,traces, edge_mapping)
            w = 1 if p1 >= p2 else -1
            #print("distribution", p1, p2)
            wp1 = p1/(p1+p2)
            wp2 = p2/(p1+p2)
            #w = majority(s,t,traces)
            scaling = 10
            entro = entropy(wp1, wp2)
            #print("entropy",wp1, wp2,entro)
            edge_cost[(s,t)] = (((1-entro) * w) -0.21 )*20
            #print(edge_cost[(s,t)])
    return edge_cost

edge_cost = compute_edge_cost(system, filtered_log, edge_mapping)

In [ ]:
def draw_dfg(g, name, layout = "sfdp"):
    # build graph with variable thicknes
    #scaling = 1/np.mean(list(nx.get_edge_attributes(g,'edge_weight').values()))

    A = to_agraph(g)
    edge_weights = nx.get_edge_attributes(g,'edge_weight')
    for e in edge_weights:
        e = A.get_edge(e[0], e[1])
        e.attr["penwidth"] = edge_weights[e]*scaling
        e.attr["fontsize"] = "20"
    for e in g.edges:
        if 'controllable' in g[e[0]][e[1]]:
            e = A.get_edge(e[0], e[1])
            if not g[e[0]][e[1]]['controllable']:
                e.attr["style"] = "dotted"
            e.attr["label"] += " count " + str(len(edge_mapping[e]))
        #A.add_edge(e[0], e[1], penwidth = edge_weights[e]*scaling)

    #A.graph_attr.update(size="7.75,10.25")
    A.write(name.split(".")[0]+".dot")
    A.layout(layout)
    print("Plotted", name)
    A.draw(name)
    
def annotate_graph(g, edge_cost):
    for e in edge_cost:
        g[e[0]][e[1]]['label'] = round(edge_cost[e],2)
    return g

def color_graph(g, green = 4, red = -4):
    g = copy.deepcopy(g)
    for e in g.edges:
        if g[e[0]][e[1]]['label'] > green:
            g[e[0]][e[1]]['color'] ="green"
        elif g[e[0]][e[1]]['label'] < red:
            g[e[0]][e[1]]['color'] ="red"
        else:
            g[e[0]][e[1]]['color'] ="gray"
    
    return g

g = annotate_graph(system, edge_cost)
g_colored = color_graph(g)
draw_dfg(g, "graph_weight.ps", "dot")
draw_dfg(g_colored, 'colored.ps', "dot")

In [ ]:
def replay(g, trace, debug = False):
    count = 0
    s = "start"
    for pos_index in range(1,len(trace)):
        action = trace[pos_index]['concept:name']
        t = -1
        for n in g[s]:
            if g[s][n]['action'] == action:
                t = n
        if (s,t) not in g.edges:
            continue
        if debug:
            print(g[s][t]['label'])
        count += g[s][t]['label']
        s = t
    return count

def replay_traces(traces, g, name, debug = False):
    weights = np.array([weight(trace) for trace in traces])
    results = np.array([replay(g,trace) for trace in traces])
    labels = np.array(["true" if w == 1 else "false" for w in weights])
    if debug:
        for i in range(len(results)):
            print("State", weights[i], "has result: ",results[i])
    traces = np.array(traces)
    if debug:
        print(np.array(weights) == 1)
    m = weights == 1

    plt.scatter(np.array(range(len(results)))[m],results[m],c = "g", label="Successfull", s = 2)
    plt.scatter(np.array(range(len(results)))[~m],results[~m],c = "r", label="Unsuccessfull", s = 2)
    plt.title("Gas per Journey")
    plt.xlabel("Journey")
    plt.ylabel("Gas")
    plt.legend()
    #plt.xticks(np.arange(0,len(results),5))
    plt.savefig(name)
    plt.show()

    traces_length = np.array([len(i) for i in traces])
    plt.scatter(np.array(range(len(results)))[m],traces_length[m],c = "g", label="Successfull", s = 2)
    plt.scatter(np.array(range(len(results)))[~m],traces_length[~m],c = "r", label="Unsuccessfull", s = 2)
    plt.title("Length of traces")
    plt.show()

replay_traces(filtered_log, g, "traces_train.png", debug = False)

In [ ]:
print(len(list(nx.simple_cycles(g_colored))))
for c in nx.simple_cycles(g_colored):
    count = 0
    c.append(c[0])
    for i in range(len(c)-1):
        count += g_colored[c[i]][c[i+1]]['label']
    print(len(c))
    print("count", count)

In [ ]:
def compute_bounds(g, start, depth):
    lower_bounds = {}
    upper_bounds = {}
    queue = [(start,0)]
    counter = 0
    while(queue and counter < depth):
        counter += 1
        s, previous_gas = queue.pop(0)
        for t in g[s]:
            current_gas = previous_gas + g[s][t]['label']
            if t not in lower_bounds:
                lower_bounds[t] = current_gas
            else:
                lower_bounds[t] = min(lower_bounds[t], current_gas)
            if t not in upper_bounds:
                upper_bounds[t] = current_gas
            else:
                upper_bounds[t] = max(upper_bounds[t], current_gas)
            queue.append((t, current_gas))
    return lower_bounds, upper_bounds
lower_bounds, upper_bounds = compute_bounds(g_colored, "start", 100)

for s in lower_bounds:
    if "SUPER" in s or "LONG" in s:
        print(s, "lower", round(lower_bounds[s]), "upper", round(upper_bounds[s]))

In [ ]:
responses_lower = []
responses_upper = []
for short in lower_bounds:
    if "SHORT" in short:
        print(short)
        current_lower = []
        current_upper = []
        long = short.replace("SHORT", "LONG")
        very = short.replace("SHORT", "SUPER LONG")
        if short in lower_bounds:
            print("short", lower_bounds[short])
            current_lower.append(lower_bounds[short])
            current_upper.append(upper_bounds[short])
        else:
            current_lower.append(0)
            current_upper.append(0)
        if long in lower_bounds:
            print("long", lower_bounds[long])
            current_lower.append(lower_bounds[long])
            current_upper.append(upper_bounds[long])
        else:
            current_lower.append(0)
            current_upper.append(0)
        if very in lower_bounds:
            print("very", lower_bounds[very])
            current_lower.append(lower_bounds[very])
            current_upper.append(upper_bounds[very])
        else:
            current_lower.append(0)
            current_upper.append(0)
        responses_lower.append(current_lower)
        responses_upper.append(current_upper)
        print("")

In [ ]:
# increase in several parts for longer communication
# in shifted model aligns upper and lower bound better
for i in range(len(responses_lower)):
    plt.plot(responses_lower[i])
    plt.plot(responses_upper[i])
    plt.show()

In [ ]:
responses_lower = []
responses_upper = []
for long in lower_bounds:
    if "LONG" in long and "SUPER" not in long:
        print(long)
        current_lower = []
        current_upper = []
        short = long.replace("LONG", "SHORT")
        very = long.replace("LONG", "SUPER LONG")
        if short in lower_bounds:
            print("short", lower_bounds[short])
            current_lower.append(lower_bounds[short])
            current_upper.append(upper_bounds[short])
        else:
            current_lower.append(0)
            current_upper.append(0)
        if long in lower_bounds:
            print("long", lower_bounds[long])
            current_lower.append(lower_bounds[long])
            current_upper.append(upper_bounds[long])
        else:
            current_lower.append(0)
            current_upper.append(0)
        if very in lower_bounds:
            print("very", lower_bounds[very])
            current_lower.append(lower_bounds[very])
            current_upper.append(upper_bounds[very])
        else:
            current_lower.append(0)
            current_upper.append(0)
        responses_lower.append(current_lower)
        responses_upper.append(current_upper)
        print("")

In [ ]:
# increase in several parts for longer communication
# in shifted model aligns upper and lower bound better
for i in range(len(responses_lower)):
    plt.plot(responses_lower[i])
    plt.plot(responses_upper[i])
    plt.show()

In [ ]:
# Computes all possible shift of lists
def shifted_lists(l):
    shifted_lists = []
    for j in range(len(l)):
        list_constructed = copy.deepcopy(l[j:])
        list_constructed.extend(l[:j])
        list_constructed.append(list_constructed[0])
        shifted_lists.append(list_constructed)
    return shifted_lists

# checks if history hist contains circle c
def contains(hist, c):
    n = len(c)+1
    max_count = 0
    lists = shifted_lists(c)
    for helper_list in lists:
        count = 0
        for i in range(len(hist)-(n-1)):
            if hist[i:i+n] == helper_list:
                count += 1
        max_count = max(max_count, count)
    return max_count

# returns true if edge (e,v) is on c
def is_on(e,v,c):
    for i in range(len(c)-1):
        if c[i] == e and c[i+1] == v:
            return True
    if c[-1] == e and c[0] == v:
        return True
    
# Presented Unrolling algorithm, Algorithm 1 with online reducing
def unroll(G, start, target, k, debug = False):
    G_gen = nx.DiGraph()
    G_gen.add_node(start, hist = [str(start)])
    if 'controllable' in G.nodes[start]:
        G_gen.nodes[start]["controllable"] = G.nodes[start]["controllable"]

    cycles = list(nx.simple_cycles(G))

    queue = [start]
    # start bf-search
    while(queue):
        if debug:
            print(len(G_gen.nodes), len(queue))
        s = queue[0]
        queue.pop(0)
        s_original = str(s).split(".")[0]
        neighbours = list(G[s_original])
        for t in neighbours:
            t_original = t
            local_hist = copy.deepcopy(G_gen.nodes[s]["hist"])
            local_hist.append(str(t_original))
            is_on_cycle = False
            can_traverse = False
            path = []
            circle = []
            relevant_cycle = []
            for c in cycles:
                if is_on(s_original,t_original,c):
                    relevant_cycle.append(c)
                    
            all_smaller = True
            for c in relevant_cycle:
                if contains(local_hist,c) >= k:
                    all_smaller = False
            
            if not all_smaller:
                paths = list(nx.all_simple_paths(G, source=t, target=target))
                for p in paths:
                    merged_hist = copy.deepcopy(local_hist)
                    merged_hist.extend(p[1:]) # 1.st element already added
                    can_not_traverse = False
                    
                    #test if no loop larger than k with path
                    for c_loop in relevant_cycle:
                        if contains(merged_hist,c_loop) > k : # check that there is path without completing additional cycle
                            can_not_traverse = True
                    can_traverse = not can_not_traverse
            if all_smaller or can_traverse:               
                #every node not on cycle can be unqiue ("merge point" within unrolled graph)
                if relevant_cycle:
                    while t in G_gen.nodes:
                        if "." not in t:
                            t += ".1"
                        else:
                            t = t.split(".")[0]+"."+str(int(t.split(".")[-1])+1)
                # add node t only to graph if not already treated

                if t not in queue:
                    queue.append(t)
                    G_gen.add_node(t, hist = local_hist)
                assert(s in G_gen and t in G_gen)
                G_gen.add_edge(s,t)
                if('label' in G[s_original][t_original]):
                    G_gen[s][t]['label'] = G[s_original][t_original]['label']
                if('controllable' in G[s_original][t_original]):
                    G_gen[s][t]['controllable'] = G[s_original][t_original]['controllable']

    print("Graph was unrolled")
    return G_gen

In [ ]:
target = [s for s in g_colored.nodes if "positive" in s or "negative" in s]
g_unroll = unroll(g_colored, "start", target, 1)

In [ ]:
# positive final states (should be one)
positives = []
for s in g_unroll.nodes:
    if "positive" in s:
        positives.append(s)
        print(s)
assert(len(positives) == 1)

In [ ]:
out_of_gas = False
def to_uppaal(g, name, layout = "sfdp"):
    f = open("/home/paul/Downloads/uppaal-4.1.20-stratego-9-linux64/"+name, "w+")
    
    pos = nx.drawing.nx_agraph.graphviz_layout(g, prog=layout, args='-Grankdir=LR')

    f.write('<?xml version="1.0" encoding="utf-8"?>')
    f.write("<!DOCTYPE nta PUBLIC '-//Uppaal Team//DTD Flat System 1.1//EN' 'http://www.it.uu.se/research/group/darts/uppaal/flat-1_1.dtd'>")
    f.write('<nta>')
    f.write('<declaration>')
    f.write('int e = 0;')
    f.write('\n'+'clock x;')
    f.write('\n'+'hybrid clock t;')
    f.write('\n'+'int steps;')
    f.write('\n'+'bool negative = false;')
    f.write('\n'+'bool positive = false;')
    f.write('\n'+'int final_gas = 300;')
    f.write('</declaration>')
    f.write('<template>')
    f.write('<name x="5" y="5">Template</name>')
    
    # print locations
    ids = {}
    for s,i in zip(pos, range(len(pos))):
        ids[s] = i
        print_location(f, "id"+str(i),pos[s][0],pos[s][1],s)
    
    max_id = max(ids.values())
    ids["outOfGas"] = max_id+1
    
    if out_of_gas:
        print_location(f,"id"+str(ids["outOfGas"]),-100,-100,"outOfGas")
    
    f.write('<init ref="id'+str(ids['start'])+'"/>')
    for e in g.edges:
        if "label" in g[e[0]][e[1]] and "controllable" in g[e[0]][e[1]]:
            print_edge(f, ids[e[0]], ids[e[1]], pos[e[0]], pos[e[1]], g[e[0]][e[1]]['label'], g[e[0]][e[1]]['controllable'], e)
        else:
            if "label" not in g[e[0]][e[1]]:
                if "controllable" not in g[e[0]][e[1]]: 
                    print_edge(f, ids[e[0]], ids[e[1]], pos[e[0]], pos[e[1]], 0, True, e)

    if out_of_gas:
        for s in g.nodes:
            print(s)
            if "positive" in str(s) or "negative" in str(s):
                continue
            print(ids[s], ids["outOfGas"])
            print_edge(f, ids[s], ids["outOfGas"], pos[s], (-100,-100), 0,True, (s,"outOfGas"), guard = False)
    f.write('</template>')
    f.write('<system>')
    f.write('Journey = Template();')
    f.write('system Journey;')
    f.write('</system>')
    f.write('</nta>')
    f.close()
    print("all written to", f.name)

def print_location(f, location_id, x, y, name):
    name = str(name)
    name = name.replace('"', '-')
    name = name.replace('{', '')
    name = name.replace('}', '')
    name = name.replace("'", '-')
    name = name.replace("_", '')
    name = name.replace("(", '')
    name = name.replace(")", '')
    f.write('<location id="'+location_id+'" x="'+str(int(x))+'" y="'+str(int(y))+'">')
    f.write('<name x="'+str(int(x))+'" y="'+str(int(y)+20)+'">'+str(name).replace(":", "").replace(" ","").replace(".", "").replace(",", "").replace("-","")+'</name>')
    f.write('<label kind="invariant" x="'+str(int(x))+'" y="'+str(int(y)-30)+'">')
    if "positive" not in name and "negative" not in name and "outOfGas" not in name:
        f.write('x &lt;= ' + str(2))
    else:
        f.write("t'==0")
    f.write('</label>')
    f.write('</location>')

def print_edge(f, s, t, pos_s, pos_t, w, controllable, e, guard = False):
    x = (pos_s[0]+pos_t[0])/2
    y = (pos_s[1]+pos_t[1])/2
    if controllable:
        f.write('<transition action = "">')
    else:
        f.write('<transition controllable="false" action = "">')
    f.write('<source ref="id'+str(s)+'"/>')
    f.write('<target ref="id'+str(t)+'"/>')
    
    if out_of_gas:
        if guard:
            f.write('<label kind="guard" x="'+str(int(x))+'" y="'+str(int(y))+'"> e&gt;= 0')
            f.write('</label>')
        else:
            f.write('<label kind="guard" x="'+str(int(x))+'" y="'+str(int(y))+'"> e&lt; 0')
            f.write('</label>')
        
    #f.write(',\n'+ 'x = 0')
    f.write(',\n'+ 'steps += 1')
    f.write(',\n'+ 'x = 0')
    if "positive" in str(e[1]):
        f.write(',\n'+ 'positive = true')
        f.write(',\n'+ 'final_gas = e +'+str(int(round(w))))
    if "negative" in str(e[1]):
        f.write(',\n'+ 'negative = true')
        f.write(',\n'+ 'final_gas = e + '+str(int(round(w))))
    f.write('<label kind="assignment" x="'+str(int(x))+'" y="'+str(int(y))+'">e = e + '+str(int(round(w))))
    f.write('</label>')
    
    if not controllable:
        if e in edge_mapping:
            f.write('<label kind="probability" x="'+str(int(x))+'" y="'+str(int(y))+'">'+str(int(len(edge_mapping[e]))))
        else:
            f.write('<label kind="probability" x="'+str(int(x))+'" y="'+str(int(y))+'">')
        f.write('</label>')
    f.write('</transition>')

to_uppaal(g_colored, "bpi2017.xml", layout = "dot")
to_uppaal(g_unroll, "bpi2017_unrolled.xml")

In [ ]:
for extension in ["SHORT", "LONG", "SUPER LONG"]:
    # partial graph implications
    states = []
    for s in g_colored.nodes:
        if "W_Call after offers "+ extension in s.split("-")[-1]:
            states.append(s)
            print(s)

    print(len(states))
    print("Full graph", len(g_colored.nodes))
    sub_nodes = set()
    for s in states:
        sub_nodes.update(set(list(nx.descendants(g_colored, s))))
        sub_nodes.add(s)
    print(len(sub_nodes))
    print(sub_nodes)
    print(len(nx.subgraph(g_colored, sub_nodes)))
    subgraph = nx.subgraph(g_colored, sub_nodes)
    subgraph = nx.DiGraph(subgraph)
    start_nodes = []
    for n in subgraph.nodes:
        if subgraph.in_degree(n) == 0:
            start_nodes.append(n)
    for n in start_nodes:
        subgraph.add_edge("start", n)
    if "start" not in subgraph.nodes:
        for n in states:
            subgraph.add_edge("start", n)
    draw_dfg(subgraph, 'subgraph_'+extension+'.ps', "dot")
    target = [s for s in subgraph.nodes if "positive" in s or "negative" in s]
    g_unroll = unroll(subgraph, "start", target, 1)
    positives = []
    for s in g_unroll.nodes:
        if "positive" in s:
            positives.append(s)
            print(s)
    assert(len(positives) <= 1)
    to_uppaal(g_unroll, "bpi2017subgraph.xml")
    out = subprocess.Popen(["/home/paul/Downloads/uppaal-4.1.20-stratego-9-linux64/bin/verifyta", "/home/paul/Downloads/uppaal-4.1.20-stratego-9-linux64/bpi2017subgraph.xml", "/home/paul/Downloads/uppaal-4.1.20-stratego-9-linux64/bpi2017_unrolled.q"], stdout=subprocess.PIPE)
    result = "is satisfied" in str(out.communicate()[0])
    print("result", result)
    results["W_Call incomplete files "+ extension] = result
print(results)
# there is difference in number of steps after interaction
# not in outcome -> longer call makes journey "clearer"
# after concept drift are journeys after calls shorter

In [ ]:
# claim: calls are not really helpful...
outcomes_call = {"pos": 0, "neg": 0}
outcomes_no_call = {"pos": 0, "neg": 0}
outcomes_short = {"pos": 0, "neg": 0}
outcomes_long = {"pos": 0, "neg": 0}
outcomes_super = {"pos": 0, "neg": 0}
for trace in filtered_log:
    call = ["W_Call after offers" in pos["concept:name"] for pos in trace]
    short = ["W_Call after offers SHORT" in pos["concept:name"] for pos in trace]
    long = ["W_Call after offers LONG" in pos["concept:name"] for pos in trace]
    super_long = ["W_Call after offers SUPER LONG" in pos["concept:name"] for pos in trace]
    result = ["positive" in pos["concept:name"] for pos in trace]
    if sum(call) > 0:
        if sum(result) > 0:
            outcomes_call["pos"] += 1
            if sum(short) > 0:
                outcomes_short["pos"] += 1
            elif sum(long) > 0:
                outcomes_long["pos"] += 1
            elif sum(super_long) > 0:
                 outcomes_super["pos"] += 1
        else:
            outcomes_call["neg"] += 1
            if sum(short) > 0:
                outcomes_short["neg"] += 1
            elif sum(long) > 0:
                outcomes_long["neg"] += 1
            elif sum(super_long) > 0:
                 outcomes_super["neg"] += 1
    else:
        if sum(result) > 0:
            outcomes_no_call["pos"] += 1
        else:
            outcomes_no_call["neg"] += 1

print("no call")
print(outcomes_no_call)
print(outcomes_no_call["pos"]/(outcomes_no_call["neg"]+outcomes_no_call["pos"]))
print("call")
print(outcomes_call)
print(outcomes_call["pos"]/(outcomes_call["neg"]+outcomes_call["pos"]))
print("short")
print(outcomes_short)
print(outcomes_short["pos"]/(outcomes_short["neg"]+outcomes_short["pos"]))
print("long")
print(outcomes_long)
print(outcomes_long["pos"]/(outcomes_long["neg"]+outcomes_long["pos"]))
print("super")
print(outcomes_super)
print(outcomes_super["pos"]/(outcomes_super["neg"]+outcomes_super["pos"]))
# calls not really impact
# after drift are super long calls 3% more effective (0.34)

In [ ]:
# call incomplete files
results = {}
for extension in ["SHORT", "LONG", "SUPER LONG"]:
    # partial graph implications
    states = []
    for s in g_colored.nodes:
        if "W_Call incomplete files "+ extension in s.split("-")[-1]:
            states.append(s)
            print(s)

    print(len(states))
    print("Full graph", len(g_colored.nodes))
    sub_nodes = set()
    for s in states:
        sub_nodes.update(set(list(nx.descendants(g_colored, s))))
        sub_nodes.add(s)
    print(len(sub_nodes))
    print(sub_nodes)
    print(len(nx.subgraph(g_colored, sub_nodes)))
    subgraph = nx.subgraph(g_colored, sub_nodes)
    subgraph = nx.DiGraph(subgraph)
    start_nodes = []
    for n in subgraph.nodes:
        if subgraph.in_degree(n) == 0:
            start_nodes.append(n)
    for n in start_nodes:
        subgraph.add_edge("start", n)
    if "start" not in subgraph.nodes:
        for n in states:
            subgraph.add_edge("start", n)
    draw_dfg(subgraph, 'subgraphg_'+extension+'.ps', "dot")
    target = [s for s in subgraph.nodes if "positive" in s or "negative" in s]
    g_unroll = unroll(subgraph, "start", target, 1)
    positives = []
    for s in g_unroll.nodes:
        if "positive" in s:
            positives.append(s)
            print(s)
    assert(len(positives) <= 1)
    to_uppaal(g_unroll, "bpi2017subgraph.xml")
    out = subprocess.Popen(["/home/paul/Downloads/uppaal-4.1.20-stratego-9-linux64/bin/verifyta", "/home/paul/Downloads/uppaal-4.1.20-stratego-9-linux64/bpi2017subgraph.xml", "/home/paul/Downloads/uppaal-4.1.20-stratego-9-linux64/bpi2017_unrolled.q"], stdout=subprocess.PIPE)
    result = "is satisfied" in str(out.communicate()[0])
    print("result", result)
    results["W_Call incomplete files "+ extension] = result
print(results)

In [ ]:
# claim: calls are not really helpful...
outcomes_call = {"pos": 0, "neg": 0}
outcomes_no_call = {"pos": 0, "neg": 0}
outcomes_short = {"pos": 0, "neg": 0}
outcomes_long = {"pos": 0, "neg": 0}
outcomes_super = {"pos": 0, "neg": 0}
for trace in filtered_log:
    call = ["W_Call incomplete files" in pos["concept:name"] for pos in trace]
    short = ["W_Call incomplete files SHORT" in pos["concept:name"] for pos in trace]
    long = ["W_Call incomplete files LONG" in pos["concept:name"] for pos in trace]
    super_long = ["W_Call incomplete files SUPER LONG" in pos["concept:name"] for pos in trace]
    result = ["positive" in pos["concept:name"] for pos in trace]
    if sum(call) > 0:
        if sum(result) > 0:
            outcomes_call["pos"] += 1
            if sum(short) > 0:
                outcomes_short["pos"] += 1
            elif sum(long) > 0:
                outcomes_long["pos"] += 1
            elif sum(super_long) > 0:
                 outcomes_super["pos"] += 1
        else:
            outcomes_call["neg"] += 1
            if sum(short) > 0:
                outcomes_short["neg"] += 1
            elif sum(long) > 0:
                outcomes_long["neg"] += 1
            elif sum(super_long) > 0:
                 outcomes_super["neg"] += 1
    else:
        if sum(result) > 0:
            outcomes_no_call["pos"] += 1
        else:
            outcomes_no_call["neg"] += 1

print("no call")
print(outcomes_no_call)
print(outcomes_no_call["pos"]/(outcomes_no_call["neg"]+outcomes_no_call["pos"]))
print("call")
print(outcomes_call)
print(outcomes_call["pos"]/(outcomes_call["neg"]+outcomes_call["pos"]))
print("short")
print(outcomes_short)
print(outcomes_short["pos"]/(outcomes_short["neg"]+outcomes_short["pos"]))
print("long")
print(outcomes_long)
print(outcomes_long["pos"]/(outcomes_long["neg"]+outcomes_long["pos"]))
print("super")
print(outcomes_super)
print(outcomes_super["pos"]/(outcomes_super["neg"]+outcomes_super["pos"]))
# calls after offers have quite the impact

In [ ]:
# partial graph implications, per activity
results = {}
print(len(set([x['concept:name'] for trace in filtered_log for x in trace])))
counter = 0
for a in set([x['concept:name'] for trace in filtered_log for x in trace]):
    print("current action", a)
    print("counter", counter)
    counter += 1
    states = []
    for s in g_colored.nodes:
        if a in s.split("-")[-1]:
            states.append(s)
            print(s)

    sub_nodes = set()
    for s in states:
        sub_nodes.update(set(list(nx.descendants(g_colored, s))))
        sub_nodes.add(s)
    subgraph = nx.subgraph(g_colored, sub_nodes)
    subgraph = nx.DiGraph(subgraph)

    start_nodes = []
    for n in subgraph.nodes:
        if subgraph.in_degree(n) == 0:
            start_nodes.append(n)
    for n in start_nodes:
        subgraph.add_edge("start", n)
    if "start" not in subgraph.nodes:
        for n in states:
            subgraph.add_edge("start", n)
    draw_dfg(subgraph, 'subgraph.ps', "dot")
    target = [s for s in subgraph.nodes if "positive" in s or "negative" in s]
    g_unroll = unroll(subgraph, "start", target, 1)
    positives = []
    for s in g_unroll.nodes:
        if "positive" in s:
            positives.append(s)
            print(s)
    assert(len(positives) <= 1)
    to_uppaal(g_unroll, "bpi2017subgraph.xml")
    out = subprocess.Popen(["/home/paul/Downloads/uppaal-4.1.20-stratego-9-linux64/bin/verifyta", "/home/paul/Downloads/uppaal-4.1.20-stratego-9-linux64/bpi2017subgraph.xml", "/home/paul/Downloads/uppaal-4.1.20-stratego-9-linux64/bpi2017_unrolled.q"], stdout=subprocess.PIPE)
    result = "is satisfied" in str(out.communicate()[0])
    print("result", result)
    results[a] = result
print(results)

In [ ]:
def draw_dfg_clustered(g, name):
    A = to_agraph(g)
    edge_weights = nx.get_edge_attributes(g,'edge_weight')
    for e in edge_weights:
        e = A.get_edge(e[0], e[1])
        e.attr["penwidth"] = edge_weights[e]*scaling
        e.attr["fontsize"] = "20"
    for e in g.edges:
        if 'controllable' in g[e[0]][e[1]]:
            e = A.get_edge(e[0], e[1])
            if not g[e[0]][e[1]]['controllable']:
                e.attr["style"] = "dotted"
            e.attr["label"] += " count " + str(len(edge_mapping[e]))
            
    for a in actors:
        print(a)
        if a not in set([x['concept:name'] for trace in filtered_log for x in trace]):
            continue
        states = []
        for s in g_colored.nodes:
            if a in s.split("-")[-1]:
                states.append(s)
        print(states)
        A.add_subgraph(states, name='cluster_'+str(a), label= a, color = "green" if results[a] else "red", fontcolor = "green" if results[a] else "red")

    A.write(name.split(".")[0]+".dot")
    A.layout("dot")
    print("Plotted", name)
    A.draw(name)
    
draw_dfg_clustered(g_colored, 'clustered.ps')

In [ ]:
states = []
for s in g_colored.nodes:
    if "O_Sent (mail and online)" in s.split("-")[-1]:
        states.append(s)
        print(s)

sub_nodes = set()
for s in states:
    sub_nodes.update(set(list(nx.descendants(g_colored, s))))
    sub_nodes.add(s)
subgraph = nx.subgraph(g_colored, sub_nodes)
subgraph = nx.DiGraph(subgraph)

start_nodes = []
for n in subgraph.nodes:
    if subgraph.in_degree(n) == 0:
        start_nodes.append(n)
for n in start_nodes:
    subgraph.add_edge("start", n)
if "start" not in subgraph.nodes:
    for n in states:
        subgraph.add_edge("start", n)
draw_dfg(subgraph, 'subgraph.ps', "dot")
target = [s for s in subgraph.nodes if "positive" in s or "negative" in s]
g_unroll = unroll(subgraph, "start", target, 1)
positives = []
for s in g_unroll.nodes:
    if "positive" in s:
        positives.append(s)
        print(s)
assert(len(positives) <= 1)
to_uppaal(g_unroll, "bpi2017subgraph.xml")
out = subprocess.Popen(["/home/paul/Downloads/uppaal-4.1.20-stratego-9-linux64/bin/verifyta", "/home/paul/Downloads/uppaal-4.1.20-stratego-9-linux64/bpi2017subgraph.xml", "/home/paul/Downloads/uppaal-4.1.20-stratego-9-linux64/bpi2017_unrolled.q"], stdout=subprocess.PIPE)
result = "is satisfied" in str(out.communicate()[0])
print("result", result)
results[a] = result